# Importing necessary libraries

In [208]:
import pandas as pd
import requests
import bs4
import re
import datetime

# Extracting the html code

In [209]:
url = "https://en.wikipedia.org/wiki/Dragons%27_Den_(British_TV_programme)#Statistics"

In [210]:
resp = requests.get(url).text

In [211]:
soup = bs4.BeautifulSoup(resp, "html.parser")

# Extracting the data

In [212]:
tables_list = soup.find_all("table",{"class":"wikitable sortable"})


In [213]:
header_list = [th.text.rstrip() for th in tables_list[1].find_all("th")]
header_list.append("Season")

In [214]:

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []  
c7 = []
c8 = []
c9 = []
c10 = []
lists = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10]



for i in range(9):# Because we have 13 tables we would like to extract and it starts with the seconde table in our list
    print(table)
    table = tables_list[i+9].find_all("tr")
    for row in table:
        cells = row.find_all("td")
        if len(cells) == 9 or len(cells) == 8:
            c1.append(cells[0].find(text=True).replace(r'Episode',''))
            c2.append(cells[1].find(text=True))
            c3.append(cells[2].find(text=True))
            c4.append(cells[3].find(text=True))
            c5.append(cells[4].find(text=True))
            if len(cells) == 8:
                c6.append(None)
                c7.append(cells[5].find(text=True))
                c8.append(cells[6].find(text=True))
                c9.append(cells[7].text)
                c10.append(9)
            else:
                c6.append(cells[5].find(text=True))
                c7.append(cells[6].find(text=True))
                c8.append(cells[7].find(text=True))
                c9.append(cells[8].text)
                c10.append(i+9)

[<tr>
<th style="width:105px">Episode
</th>
<th style="width:105px">First aired
</th>
<th style="width:105px">Entrepreneur(s)
</th>
<th style="width:150px">Company or product name
</th>
<th style="width:70px">Money requested (£)
</th>
<th style="width:20px">%
</th>
<th style="width:400px">Description of product
</th>
<th style="width:200px">Investing Dragon(s)
</th>
<th style="width:70px">Website and Fate
</th></tr>, <tr>
<td>Episode 1
</td>
<td>11 August 2019
</td>
<td>Tom Keen
</td>
<td>Flush Brush
</td>
<td style="text-align:center;">50,000
</td>
<td style="text-align:center;">50%<sup class="reference" id="cite_ref-90"><a href="#cite_note-90">[note 1]</a></sup>
</td>
<td>Alternative Toilet Cleaner
</td>
<td>Touker Suleyman and Sara Davies
</td>
<td style="text-align:center;"><a class="external autonumber" href="https://www.flushbrush.co.uk/" rel="nofollow">[196]</a>
</td></tr>, <tr>
<td>Episode 1
</td>
<td>11 August 2019
</td>
<td>Katy Foxcroft and Gillian Robson
</td>
<td>Tancream


In [215]:
dictionnary = {i:j for i,j in zip(header_list, lists)}

In [216]:
df_shark_tank_table = pd.DataFrame(dictionnary)

In [217]:
df_shark_tank_table = df_shark_tank_table.replace(r'\n','', regex=True).replace(r"\[\d+]",'', regex=True)

## Getting current GBP to USD exchange rate from api

In [218]:
rate_GBP_USD = requests.get('https://api.exchangeratesapi.io/latest?base=GBP&symbols=USD')

In [219]:
resp_data = rate_GBP_USD.json()
resp_data

{'rates': {'USD': 1.3190324563}, 'base': 'GBP', 'date': '2020-08-27'}

In [220]:
rate = resp_data['rates']['USD']

## Converting the column Money requested in £ to $

In [221]:
df_shark_tank_table["Money requested (£)"] = df_shark_tank_table["Money requested (£)"].replace(r",", "", regex=True).astype(float)

In [222]:
df_shark_tank_table["Money requested (£)"] = round(df_shark_tank_table["Money requested (£)"] * rate).astype(int)

## Changing the name of the columns to fit the naming convention

In [223]:
df_shark_tank_table = df_shark_tank_table.rename(columns={"Money requested (£)": "money_requested_converted_$", "%" : "stake_in_%_sold", "Website and Fate" : "fate"})

In [224]:
df_shark_tank_table.columns = df_shark_tank_table.columns.str.lower().str.replace(r" ", "_")

## Deleting everything in parentheses in the fate column

In [225]:
df_shark_tank_table["fate"] = df_shark_tank_table["fate"].replace(r'\(.+\)','', regex=True)
df_shark_tank_table.loc[df_shark_tank_table['fate'] == "", 'fate'] = 'active'
df_shark_tank_table.head()

,episode,first_aired,entrepreneur(s),company_or_product_name,money_requested_converted_$,stake_in_%_sold,description_of_product,investing_dragon(s),fate,season
0,1,31 July 2011,Georgette Hewitt,The Present Club,79142,None,A website for buying gifts for children,Peter Jones and Theo Paphitis,active,9
1,1,31 July 2011,Chris Hopkins,Ploughcroft Solar,158284,None,A company that installs solar panels in homes,Deborah Meaden and Theo Paphitis,active,9
2,2,7 August 2011,"Christian Hartmann, Martin McLaughlin, Tom Cal...",Love Da Pop,92332,None,A business turning the old popcorn treat into ...,Peter Jones,active,9
3,2,7 August 2011,Liz and Alan Colleran,Raskelf Memory Foam (Duvalay),105523,None,A zip-up memory foam mattress and duvet combo,Hilary Devey,active,9
4,3,14 August 2011,Peter and Michelle Hart,Fun Fancy Dress,131903,None,Fancy-dress shop business and franchise,Duncan Bannatyne,\tdeal fell through after filming,9


## In stake_in_%_sold deleting % and transfroming column to int

In [226]:
df_shark_tank_table["stake_in_%_sold"] = df_shark_tank_table["stake_in_%_sold"].replace(r'%','', regex=True).str.strip()

In [227]:
df_shark_tank_table["stake_in_%_sold"] = pd.to_numeric(df_shark_tank_table["stake_in_%_sold"], errors='coerce')

In [228]:
df_shark_tank_table["valuation"] = round((df_shark_tank_table['money_requested_converted_$']/df_shark_tank_table["stake_in_%_sold"])*100).astype('Int32')

In [229]:
df_shark_tank_table

,episode,first_aired,entrepreneur(s),company_or_product_name,money_requested_converted_$,stake_in_%_sold,description_of_product,investing_dragon(s),fate,season,valuation
0,1,31 July 2011,Georgette Hewitt,The Present Club,79142,NaN,A website for buying gifts for children,Peter Jones and Theo Paphitis,active,9,<NA>
1,1,31 July 2011,Chris Hopkins,Ploughcroft Solar,158284,NaN,A company that installs solar panels in homes,Deborah Meaden and Theo Paphitis,active,9,<NA>
2,2,7 August 2011,"Christian Hartmann, Martin McLaughlin, Tom Cal...",Love Da Pop,92332,NaN,A business turning the old popcorn treat into ...,Peter Jones,active,9,<NA>
3,2,7 August 2011,Liz and Alan Colleran,Raskelf Memory Foam (Duvalay),105523,NaN,A zip-up memory foam mattress and duvet combo,Hilary Devey,active,9,<NA>
4,3,14 August 2011,Peter and Michelle Hart,Fun Fancy Dress,131903,NaN,Fancy-dress shop business and franchise,Duncan Bannatyne,\tdeal fell through after filming,9,<NA>
...,...,...,...,...,...,...,...,...,...,...,...
133,10,15 March 2020,David and Julie Gray,Viper Clip,105523,25.0,Fully Insulated Cable Staple,Deborah Meaden,active,17,422092
134,11,22 March 2020,Jess and Philip,Brain Fud Drinks,65952,30.0,Natural Energy Drinks,Peter Jones,active,17,219840
135,11,22 March 2020,Ross Lamond,Bug Bakes,65952,35.0,Insect Protein Dog Food,Touker Suleyman,active,17,188434
136,12,29 March 2020,Eric and Hugo,Lemuro,98927,25.0,Phone Camera Lenses,Deborah Meaden,active,17,395708


# Exporting the DataFrame as a .csv file

In [230]:
df_shark_tank_table.to_csv('Dragons_den_web_scrape_wikipedia.csv', index = False)